In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
sns.set()

In [ ]:
df = pd.read_csv('data/apple_stock_short.csv', parse_dates=['Date'])
df.set_index('Date', inplace=True)
df = df.asfreq('B')
df['Close'].interpolate(inplace=True)
df.head()

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(df.index, df['Close'])
plt.title('AAPL - Close Prices (sample)')
plt.xlabel('Date')
plt.ylabel('Close')
plt.tight_layout()
plt.savefig('screenshots/price_plot.png')
plt.show()

In [ ]:
decomp = sm.tsa.seasonal_decompose(df['Close'], model='additive', period=20)
fig = decomp.plot()
fig.set_size_inches(10,6)
plt.tight_layout()
plt.savefig('screenshots/trend_decomposition.png')
plt.show()

In [ ]:
result = adfuller(df['Close'].dropna())
print('ADF Statistic:', result[0])
print('p-value:', result[1])

In [ ]:
plt.figure(); plot_acf(df['Close'].dropna(), lags=30); plt.savefig('screenshots/acf.png'); plt.show()
plt.figure(); plot_pacf(df['Close'].dropna(), lags=30, method='ywm'); plt.savefig('screenshots/pacf.png'); plt.show()

In [ ]:
train = df['Close'][:-30]
test = df['Close'][-30:]
model = sm.tsa.ARIMA(train, order=(1,1,1)).fit()
print(model.summary())
fc = model.get_forecast(steps=30)
fc_mean = fc.predicted_mean
ci = fc.conf_int()
plt.figure(figsize=(10,4))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test')
plt.plot(fc_mean.index, fc_mean, label='Forecast')
plt.fill_between(ci.index, ci.iloc[:,0], ci.iloc[:,1], color='k', alpha=0.1)
plt.legend()
plt.title('ARIMA Forecast')
plt.savefig('screenshots/forecast.png')
plt.show()